In [1]:
import pandas as pd
import numpy as np

In [2]:
# 서부텍사스 중질유

In [3]:
df1 = pd.read_excel('./datasets/11.2 WTI.xlsx','Spot', index_col='Date')  # 현물
df2 = pd.read_excel('./datasets/11.2 WTI.xlsx','Front', index_col='Date') # 최근월물, 만기를 앞두고 있는 선물
df3 = pd.read_excel('./datasets/11.2 WTI.xlsx','Second', index_col='Date')# 차근월물, 다음 주기에 만기가 도래할 선물

# Data load

In [4]:
df = pd.concat([df1,df2,df3], axis = 1)
df.head()

,Spot,Front,Second
Date,,,
2017-12-28,59.84,59.84,59.87
2017-12-27,59.64,59.64,59.69
2017-12-26,59.97,59.97,60.00
2017-12-22,58.42,58.47,58.54
2017-12-21,58.33,58.36,58.40


In [5]:
df = df.fillna(method='ffill')
df = df.fillna(method='bfill')

In [6]:
df_e = pd.read_excel('./datasets/11.2 WTI.xlsx','Expiry')
df_e[120:125]

,Ticker,Month,Last Trade,Last Price
120,CLG10,2017-02-10,01/20/10,77.62
121,CLH10,2017-03-10,02/22/10,80.16
122,CLJ10,2017-04-10,03/22/10,81.25
123,CLK10,2017-05-10,04/20/10,83.45
124,CLM10,2017-06-10,05/20/10,68.01


# 만기도래

In [7]:
# 만기일 추출
df_e['Last Trade'].head()

0    01/20/00
1    02/22/00
2    03/21/00
3    04/19/00
4    05/22/00
Name: Last Trade, dtype: object

In [8]:
expiry = pd.Series(pd.to_datetime(df_e["Last Trade"]))
expiry.head()

0   2000-01-20
1   2000-02-22
2   2000-03-21
3   2000-04-19
4   2000-05-22
Name: Last Trade, dtype: datetime64[ns]

In [9]:
df['Expiry'] = np.nan
df.tail()

,Spot,Front,Second,Expiry
Date,,,,
2010-01-08,82.75,82.75,83.30,NaN
2010-01-07,82.66,82.66,83.19,NaN
2010-01-06,83.18,83.18,83.75,NaN
2010-01-05,81.77,81.77,82.41,NaN
2010-01-04,81.51,81.51,82.12,NaN


In [10]:
# trading day 중 만기일 찾아서 표시하기
for d in df.index:
    if d in pd.to_datetime(expiry.values):
        df.loc[d].Expiry = 1
df[5:10]

,Spot,Front,Second,Expiry
Date,,,,
2017-12-20,58.02,58.09,58.13,NaN
2017-12-19,57.46,57.46,57.56,1.0
2017-12-18,57.16,57.16,57.22,NaN
2017-12-15,57.30,57.30,57.33,NaN
2017-12-14,57.04,57.04,57.08,NaN


# 손익 계산

## 현물에 투자 
- 손익 = 현재가격-초기가격

In [11]:
df.loc['2017-12-28','Spot']

59.84

In [12]:
price_now    = float(df.loc['2017-12-28','Spot']) # 현물 현재가격
price_before = float(df.loc['2010-01-04','Spot']) # 현물 최초가격
profit = (price_now - price_before) * 1000
print('손익(현재가격 - 초기가격) : ', '{:,.2f}'.format(profit))

손익(현재가격 - 초기가격) :  -21,670.00


## 선물에 투자
* 현재가격 - 초기가격 - 롤오버가격

In [16]:
len(df[df['Expiry']==1])

96

In [13]:
# 서부텍사스 중질유는 매달 만기 -> 매달 롤오버
# 롤오버 비용이 손익에 반영되어야 함
# 롤오버 비용 = 최근월물을 팔고 차근월물을 매입하는 가격
# second - first

In [14]:
roll_over_cost_acc = 0 # 롤오버 비용 누적값 초기화
for d in df.index:
    if (df.loc[d,'Expiry']==1):
        roll_over_cost = df.loc[d,'Second'] - df.loc[d,'Front']
        roll_over_cost_acc = roll_over_cost_acc + roll_over_cost
print('누적 롤오버 비용 :', '{:,.2f}'.format(roll_over_cost_acc))

누적 롤오버 비용 : 40.68


In [15]:
df.loc['2017-12-19','Second']

57.56

In [17]:
price_now    = float(df.loc['2017-12-28','Front']) # 현물 현재가격
price_before = float(df.loc['2010-01-04','Front'])
profit = (price_now - price_before - roll_over_cost_acc) * 1000
print('누적비용 : ', '{:,.2f}'.format(profit))

누적비용 :  -62,350.00
